<a href="https://colab.research.google.com/github/Kolawole-a2/Kola_Projects/blob/main/MITRE_TABLE_MERGED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- FIX CELL: robust D3FEND loader + re-run merge + download ZIP ---

import os, re, io, zipfile
import pandas as pd
import requests
from google.colab import files

# Reuse your existing ATT&CK CSV if it's already created
ATTACK_CSV = "/content/attack_techniques.csv"

# Endpoints (CSV first because it's more stable)
D3FEND_JSON_URL = "https://d3fend.mitre.org/api/ontology/inference/d3fend-full-mappings.json"
D3FEND_CSV_URL  = "https://d3fend.mitre.org/api/ontology/inference/d3fend-full-mappings.csv"

def fetch_json(url: str):
    r = requests.get(url, timeout=60, headers={"User-Agent":"colab-mitre-builder"})
    r.raise_for_status()
    return r.json()

def fetch_text(url: str) -> str:
    r = requests.get(url, timeout=60, headers={"User-Agent":"colab-mitre-builder"})
    r.raise_for_status()
    return r.text

def _extract_all_tids_from_value(val) -> list:
    """Find all ATT&CK technique IDs in a text value (T#### or T####.###)."""
    if pd.isna(val):
        return []
    s = str(val)
    return re.findall(r"(T\d{4}(?:\.\d{3})?)", s)

def _first_nonempty(*vals):
    for v in vals:
        if isinstance(v, str) and v.strip():
            return v.strip()
    return ""

def _infer_category_from_name(name: str) -> str:
    key = (name or "").lower()
    for cat in ["harden","isolate","detect","deceive","evict","neutralize","recover","resilience","respond"]:
        if cat in key:
            return cat.capitalize()
    return ""

def load_d3fend_schema_agnostic() -> pd.DataFrame:
    """
    Try CSV first (cleaner), then JSON. Build a tidy table with columns:
      attack_technique_id, d3fend_name, d3fend_category, relation, d3fend_raw
    by scanning all columns for ATT&CK IDs and D3FEND terms.
    """
    def from_csv() -> pd.DataFrame:
        txt = fetch_text(D3FEND_CSV_URL)
        raw = pd.read_csv(io.StringIO(txt))
        if raw.empty:
            return raw

        rows = []
        lower_cols = [c.lower() for c in raw.columns]

        # Heuristics to pick possible columns (they vary by release)
        # Prefer explicit columns if present
        attack_id_like = [c for c in raw.columns if "technique id" in c.lower() and "attack" in c.lower()]
        attack_name_like = [c for c in raw.columns if "technique name" in c.lower() and "attack" in c.lower()]
        d3_name_like = [c for c in raw.columns if "d3fend" in c.lower() and "name" in c.lower()]
        d3_cat_like  = [c for c in raw.columns if "category" in c.lower() or "tactic" in c.lower()]
        relation_like = [c for c in raw.columns if "relation" in c.lower() or "predicate" in c.lower()]

        # If explicit columns exist, use them directly and still extract TIDs as a safety net
        for _, r in raw.iterrows():
            tids = set()
            if attack_id_like:
                for col in attack_id_like:
                    tids.update(_extract_all_tids_from_value(r[col]))
            # If none found, sweep all cells
            if not tids:
                for v in r.to_list():
                    for tid in _extract_all_tids_from_value(v):
                        tids.add(tid)

            if not tids:
                continue

            d3_name = _first_nonempty(*(r[c] for c in d3_name_like)) if d3_name_like else ""
            # If no explicit D3FEND name col, try to sniff any field containing 'd3f:' or 'd3fend'
            if not d3_name:
                candidate = ""
                for c in raw.columns:
                    v = str(r[c])
                    if "d3f:" in v or "d3fend" in v.lower():
                        candidate = v
                        break
                d3_name = candidate.split(":")[-1] if ":" in candidate else candidate

            d3_cat  = _first_nonempty(*(r[c] for c in d3_cat_like)) if d3_cat_like else _infer_category_from_name(d3_name)
            relation = _first_nonempty(*(r[c] for c in relation_like)) if relation_like else ""

            for tid in tids:
                rows.append({
                    "attack_technique_id": tid,
                    "d3fend_name": str(d3_name),
                    "d3fend_category": str(d3_cat),
                    "relation": str(relation),
                    "d3fend_raw": str(d3_name),  # keep raw name here as well
                })

        df = pd.DataFrame(rows).drop_duplicates()
        return df

    def from_json() -> pd.DataFrame:
        data = fetch_json(D3FEND_JSON_URL)
        # Common shapes: {'results': {'bindings': [...]}} or {'bindings': [...]}
        bindings = data.get("results", {}).get("bindings", []) or data.get("bindings", []) or []
        rows = []
        def label(d):
            return d.get("label") or d.get("value") or ""
        for b in bindings:
            subj = label(b.get("s", {})) or label(b.get("subject", {}))
            pred = label(b.get("p", {})) or label(b.get("predicate", {}))
            obj  = label(b.get("o", {})) or label(b.get("object", {}))

            # extract any TIDs mentioned anywhere in this triple
            tids = set(_extract_all_tids_from_value(" ".join([subj, pred, obj])))
            if not tids:
                continue

            # try to capture a D3FEND-ish label from subj/obj
            d3_name = ""
            for t in (subj, obj):
                if "d3f:" in t or "d3fend" in t.lower():
                    d3_name = t.split(":")[-1] if ":" in t else t
                    break
            relation = pred or ""
            d3_cat = _infer_category_from_name(d3_name)
            for tid in tids:
                rows.append({
                    "attack_technique_id": tid,
                    "d3fend_name": d3_name,
                    "d3fend_category": d3_cat,
                    "relation": relation,
                    "d3fend_raw": d3_name,
                })
        return pd.DataFrame(rows).drop_duplicates()

    # Try CSV first; if it fails, try JSON.
    try:
        df = from_csv()
        if not df.empty:
            return df
    except Exception as e:
        print("CSV load failed, will try JSON. Reason:", e)

    try:
        df = from_json()
        return df
    except Exception as e:
        print("JSON load failed as well:", e)
        return pd.DataFrame(columns=["attack_technique_id","d3fend_name","d3fend_category","relation","d3fend_raw"])

# ---------- Load assets and merge ----------

# 1) Load ATT&CK techniques (already created in your previous run)
if not os.path.exists(ATTACK_CSV):
    raise FileNotFoundError(f"{ATTACK_CSV} not found. Re-run the first cell to create it.")

df_attack = pd.read_csv(ATTACK_CSV)

# 2) Load D3FEND (schema-agnostic)
print("Loading D3FEND mappings (schema-agnostic)…")
df_d3 = load_d3fend_schema_agnostic()
print("D3FEND rows:", len(df_d3))
if df_d3.empty:
    raise RuntimeError("D3FEND mappings appear empty. Try re-running in a minute or check connectivity.")

# 3) Normalize technique IDs and merge
def norm_tid(x):
    if pd.isna(x): return None
    m = re.search(r"(T\d{4}(?:\.\d{3})?)", str(x))
    return m.group(1) if m else None

df_d3["attack_technique_id"] = df_d3["attack_technique_id"].map(norm_tid)
df_d3 = df_d3.dropna(subset=["attack_technique_id"]).drop_duplicates()

# If df_attack tactics/platforms are lists stored as strings, keep them as-is; merge by ID
keep_cols = ["technique_id","technique_name","is_subtechnique","parent_technique_id","tactics","platforms"]
available_keep = [c for c in keep_cols if c in df_attack.columns]

merged = df_d3.merge(
    df_attack[available_keep],
    left_on="attack_technique_id",
    right_on="technique_id",
    how="left"
)

col_order = [
    "attack_technique_id","technique_name","is_subtechnique","parent_technique_id","tactics","platforms",
    "d3fend_name","d3fend_category","relation","d3fend_raw"
]
for c in col_order:
    if c not in merged.columns:
        merged[c] = ""

merged = merged[col_order].drop_duplicates().reset_index(drop=True)

# 4) Save & download
attack_out = "/content/attack_techniques.csv"          # overwrite with same path for convenience
merged_out = "/content/attack_d3fend_merged.csv"
zip_out    = "/content/mitre_attack_d3fend_tables.zip"

df_attack.to_csv(attack_out, index=False)
merged.to_csv(merged_out, index=False)

with zipfile.ZipFile(zip_out, "w", zipfile.ZIP_DEFLATED) as zf:
    zf.write(attack_out, arcname=os.path.basename(attack_out))
    zf.write(merged_out, arcname=os.path.basename(merged_out))

print(f"Saved {attack_out} ({len(df_attack)} rows)")
print(f"Saved {merged_out} ({len(merged)} rows)")

files.download(zip_out)


Loading D3FEND mappings (schema-agnostic)…
D3FEND rows: 1472
Saved /content/attack_techniques.csv (679 rows)
Saved /content/attack_d3fend_merged.csv (1472 rows)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>